## packages

In [104]:
# import libraries
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

## website scraping

In [116]:
shipsDate = []
for i in range(1,776):
    #quote_page = f"https://archiveofourown.org/tags/Percy%20Jackson%20and%20the%20Olympians%20*a*%20Related%20Fandoms%20-%20All%20Media%20Types/works?commit=Sort+and+Filter&page={i}&utf8=%E2%9C%93&work_search%5Bcomplete%5D=&work_search%5Bcrossover%5D=F&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Blanguage_id%5D=&work_search%5Bother_tag_names%5D=&work_search%5Bquery%5D=&work_search%5Bsort_column%5D=revised_at&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D="
    t0 = time.time()
    time.sleep(5)
    page = requests.get(quote_page)
    response_delay = time.time() - t0
    soup = BeautifulSoup(page.content, "html.parser")
    
    date = soup.find('p', attrs={'class': 'datetime'})
    date = date.text.strip()
    
    ships = soup.find_all('li', attrs={'class': 'relationships'})
    
    shipsList = []
    for i in range(0,len(ships)):
        shipsList.append(ships[i].text.strip())

    for i in shipsList:
        shipsDate.append([i,date])
    

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [132]:
df = pd.DataFrame(shipsDate, columns= ["Ship","Date"])

In [119]:
df.to_csv('output1.csv')

## start here after scraping

In [142]:
df = pd.read_csv('output1.csv')

In [143]:
pd.isnull(df).any(1).nonzero()[0]

array([], dtype=int64)

In [144]:
df = df[df['Ship'].str.contains('/')]

In [145]:
df['Ship'] = df['Ship'].str.replace(r"\(.*\)","")

In [146]:
individuals = df["Ship"].str.split("/", expand = True) 

personColumns = []
for i in range(0,7):
    df[f"Person {i+1}"] = individuals[i]
    personColumns.append(f"Person {i+1}")

In [147]:
df.head()

,Ship,Date,Person 1,Person 2,Person 3,Person 4,Person 5,Person 6,Person 7
0,Annabeth Chase/Percy Jackson,02 Apr 2019,Annabeth Chase,Percy Jackson,None,None,None,None,None
1,Nico di Angelo/Pollux,02 Apr 2019,Nico di Angelo,Pollux,None,None,None,None,None
2,Annabeth Chase/Percy Jackson,02 Apr 2019,Annabeth Chase,Percy Jackson,None,None,None,None,None
4,Annabeth Chase/Piper McLean,02 Apr 2019,Annabeth Chase,Piper McLean,None,None,None,None,None
5,Aphrodite/Ares,02 Apr 2019,Aphrodite,Ares,None,None,None,None,None


In [148]:
dfAnnabeth = df[df['Person 1'].str.contains("Annabeth")].copy()
dfAnnabeth['Person 1'].unique()
dfAnnabeth.loc[dfAnnabeth['Person 1'].str.contains('Annabeth', case=False), 'Person 1'] = 'Annabeth Chase'

In [149]:
mainCharacters = ['Percy', 'Annabeth', 'Grover', 'Leo' , 'Nico', 'Jason', 'Thalia', 
                  'Rachel', 'Bianca', 'Calypso' ,'Charles', 'Clarisse','Ethan',
                 'Juniper','Luke','Selena','Zo','Tyson','Will','Travis','Connor',
                 'Frank','Hazel','Octavian','Piper','Reyna','Magnus','Sam','Sally','Silena','Chris',
                  'Hylla','Drew','Frederick','Alex','Katie','Lou']
gods = ['Aphrodite','Ares','Athena','Chiron','Poseidon','Hera',
        'Hephaestus','Demeter','Zeus','Hades','Apollo','Artemis','Hestia','Hedge',
        'Persephone', 'Diana']

In [150]:
for i in personColumns:
    for j in range(0,len(mainCharacters)):
        df.loc[df[i].str.contains(mainCharacters[j], case=False, na=False), i] = mainCharacters[j]
        
    for k in range(0,len(gods)):
        df.loc[df[i].str.contains(gods[k], case=False, na=False), i] = gods[k]

    df.loc[df[i].str.contains('Original', case=False, na=False), i] = 'OC'

    df.loc[df[i].str.contains('Reader', case=False, na=False), i] = 'Reader'
    df.loc[df[i].str.contains('you', case=False, na=False), i] = 'Reader'

In [156]:
value_counts = df[personColumns].stack().value_counts()
#print(value_counts)

In [154]:
threshold = 50
toRemove = value_counts[value_counts <= threshold].index
for i in personColumns:
    df[i].replace(toRemove, np.nan, inplace=True)

In [155]:
value_counts = df[personColumns].stack().value_counts()
print(value_counts)

Nico           3542
Percy          3433
Will           2442
Annabeth       1951
Jason          1828
Piper          1093
Leo             838
Frank           678
Hazel           659
Reyna           451
Calypso         390
Luke            358
OC              309
Thalia          298
Reader          124
Sally           124
Connor          123
Apollo          119
Hades           102
Travis          102
Poseidon         97
Rachel           97
Silena           90
Grover           90
Mitchell         86
Clarisse         84
Octavian         81
Paul Blofis      80
Zeus             79
Juniper          75
Charles          73
Ethan            70
Drew             65
Katie            64
Alex             51
Chris            51
dtype: int64
